# <font color='redaqua'> Portfolio Risk Management


## What is Portfolio Risk Management?

Portfolio risk management is the process of identifying, assessing, and prioritizing risks to an investment portfolio. The goal is to maximize returns while minimizing the likelihood and impact of negative events.
Some common metrics used in portfolio risk management include:

* Value at Risk (VaR)
* Expected Shortfall (ES)
* Conditional Value at Risk (CVaR)
* Maximum Drawdown
* Beta
* Alpha
* Sharpe Ratio
* Treynor Ratio
* Sortino Ratio
* Correlation coefficient
* Standard deviation
* Covariance
* Correlation matrix
* Scenario analysis
* Stress testing

These metrics are used to quantify and measure the potential losses and returns of a portfolio. It's important to note that these are only a subset of all available metrics and different ones may be more suitable for different types of portfolios or investment strategies

In [49]:
# Import Modules
import pandas as pd
import numpy as np
import yfinance as yf
import statsmodels.api as sm
from scipy.stats import norm
from scipy.optimize import minimize
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression
import warnings

warnings.filterwarnings("ignore")


In [50]:
# Download the data
stock_list = ['AAPL', 'MSFT','AMZN', 'NVDA', 'KO', 'MCD', 'NFLX', 'NKE']
benchmark = "^GSPC"
start_date = "2017-01-01" 
end_date = "2021-01-01"
port_weights = np.random.rand(len(stock_list))
port_weights /= np.sum(port_weights)
df = yf.download(stock_list, start_date, end_date)["Adj Close"]
df.head()

[*********************100%***********************]  8 of 8 completed


,AAPL,AMZN,KO,MCD,MSFT,NFLX,NKE,NVDA
Date,,,,,,,,
2017-01-03 00:00:00-05:00,27.174755,37.683498,34.493881,103.605553,57.645374,127.489998,48.821358,25.147223
2017-01-04 00:00:00-05:00,27.144331,37.859001,34.370102,103.484299,57.387447,129.410004,49.845131,25.733931
2017-01-05 00:00:00-05:00,27.282379,39.022499,34.452625,103.674843,57.387447,131.809998,49.835739,25.080662
2017-01-06 00:00:00-05:00,27.586531,39.799500,34.444370,104.592934,57.884865,131.070007,50.634087,25.415924
2017-01-09 00:00:00-05:00,27.839207,39.846001,34.097786,104.307098,57.700630,130.949997,50.136284,26.446367


In [51]:
# Plotting the prices
fig = go.Figure()

for col in df.columns:
    fig.add_trace(go.Scatter(x = df.index, y = df[col], name = col))

fig.show()


In [52]:
df_rtn = df.pct_change()
df_rtn.head()

,AAPL,AMZN,KO,MCD,MSFT,NFLX,NKE,NVDA
Date,,,,,,,,
2017-01-03 00:00:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-04 00:00:00-05:00,-0.001120,0.004657,-0.003588,-0.001170,-0.004474,0.015060,0.020970,0.023331
2017-01-05 00:00:00-05:00,0.005086,0.030732,0.002401,0.001841,0.000000,0.018546,-0.000188,-0.025386
2017-01-06 00:00:00-05:00,0.011148,0.019912,-0.000240,0.008855,0.008668,-0.005614,0.016020,0.013367
2017-01-09 00:00:00-05:00,0.009159,0.001168,-0.010062,-0.002733,-0.003183,-0.000916,-0.009831,0.040543


In [53]:
fig = go.Figure()

for col in df_rtn.columns:
    fig.add_trace(go.Histogram(x = df_rtn[col], name = col))

fig.update_layout(title = "Stock Returns Histogram", xaxis_title = "Returns",
                  yaxis_title = "Frequency")
fig.show()

In [54]:
pf1_cum_rtn = (1 + df_rtn).cumprod()

In [55]:
# Plotting the prices
fig = go.Figure()

for col in df.columns:
    fig.add_trace(go.Scatter(x = pf1_cum_rtn.index, y = pf1_cum_rtn[col], name = col))

fig.show()


In [56]:
pf1_rtn_weights = (df_rtn * port_weights).sum(axis = 1)

In [57]:
pf1_rtn = pd.DataFrame()
pf1_rtn["Returns"] = pf1_rtn_weights
pf1_rtn.tail()

,Returns
Date,
2020-12-24 00:00:00-05:00,0.002698
2020-12-28 00:00:00-05:00,0.016988
2020-12-29 00:00:00-05:00,0.000024
2020-12-30 00:00:00-05:00,-0.005271
2020-12-31 00:00:00-05:00,0.003450


In [58]:
pf1_cum_rtn = (1 + pf1_rtn).cumprod()
pf1_cum_rtn.tail()

,Returns
Date,
2020-12-24 00:00:00-05:00,3.603528
2020-12-28 00:00:00-05:00,3.664745
2020-12-29 00:00:00-05:00,3.664835
2020-12-30 00:00:00-05:00,3.645519
2020-12-31 00:00:00-05:00,3.658096


In [59]:
fig = go.Figure()

for col in pf1_rtn.columns:
    fig.add_trace(go.Histogram(x = pf1_rtn[col], name = col))

fig.update_layout(title = "Portfolio Returns Histogram", xaxis_title = "Returns",
                  yaxis_title = "Frequency")
fig.show()

In [60]:
# Plotting the prices
fig = go.Figure()

fig.add_trace(go.Scatter(x = pf1_cum_rtn.index, y = pf1_cum_rtn["Returns"], name = "Returns"))

fig.show()


In [61]:
pf1_cov_matrix = df_rtn.cov()
pf1_vol = np.sqrt(np.dot(port_weights.T, np.dot(pf1_cov_matrix, port_weights)))
pf1_cov_matrix

,AAPL,AMZN,KO,MCD,MSFT,NFLX,NKE,NVDA
AAPL,0.000398,0.000246,0.000107,0.000140,0.000271,0.000243,0.000181,0.000368
AMZN,0.000246,0.000372,0.000059,0.000086,0.000254,0.000306,0.000129,0.000338
KO,0.000107,0.000059,0.000178,0.000106,0.000113,0.000048,0.000116,0.000113
MCD,0.000140,0.000086,0.000106,0.000241,0.000148,0.000090,0.000148,0.000175
MSFT,0.000271,0.000254,0.000113,0.000148,0.000331,0.000258,0.000180,0.000364
NFLX,0.000243,0.000306,0.000048,0.000090,0.000258,0.000621,0.000135,0.000380
NKE,0.000181,0.000129,0.000116,0.000148,0.000180,0.000135,0.000346,0.000243
NVDA,0.000368,0.000338,0.000113,0.000175,0.000364,0.000380,0.000243,0.000897


In [62]:
pf1_vol

0.014542448916940839

In [63]:
pf1_vol_ann = pf1_vol * np.sqrt(252)
pf1_vol_ann

0.23085421972851636

In [64]:
sp500 = yf.download(benchmark, start_date, end_date)["Adj Close"]
sp500_rtn = sp500.pct_change()
sp500_cum_rtn = (1 + sp500_rtn).cumprod().dropna()
sp500_cum_rtn

[*********************100%***********************]  1 of 1 completed


Date
2017-01-04 00:00:00-05:00    1.005722
2017-01-05 00:00:00-05:00    1.004947
2017-01-06 00:00:00-05:00    1.008482
2017-01-09 00:00:00-05:00    1.004903
2017-01-10 00:00:00-05:00    1.004903
                               ...   
2020-12-24 00:00:00-05:00    1.640097
2020-12-28 00:00:00-05:00    1.654403
2020-12-29 00:00:00-05:00    1.650718
2020-12-30 00:00:00-05:00    1.652932
2020-12-31 00:00:00-05:00    1.663575
Name: Adj Close, Length: 1006, dtype: float64

In [65]:
df2 = pd.merge(pf1_cum_rtn, sp500_cum_rtn, on = "Date", how = "inner")
df2.rename(columns = {"Adj Close": "SP500", "Returns": "Portfolio"}, inplace = True)
df2.head()

,Portfolio,SP500
Date,,
2017-01-04 00:00:00-05:00,1.004882,1.005722
2017-01-05 00:00:00-05:00,1.011264,1.004947
2017-01-06 00:00:00-05:00,1.020207,1.008482
2017-01-09 00:00:00-05:00,1.020136,1.004903
2017-01-10 00:00:00-05:00,1.016969,1.004903


In [66]:
# Plotting the prices
fig = go.Figure()

for col in df2.columns:
    fig.add_trace(go.Scatter(x = df2.index, y = df2[col], name = col))

fig.update_layout(title = "Portfolio x SP500")
fig.show()


In [67]:
# Calculating the Beta
Y = df2["Portfolio"]
X = df2["SP500"]

X = sm.add_constant(X)
regression = sm.OLS(Y, X)
beta = regression.fit()
beta = beta.params[1]
beta 


4.204494847174073

In [68]:
pf1_rtn

,Returns
Date,
2017-01-03 00:00:00-05:00,0.000000
2017-01-04 00:00:00-05:00,0.004882
2017-01-05 00:00:00-05:00,0.006350
2017-01-06 00:00:00-05:00,0.008843
2017-01-09 00:00:00-05:00,-0.000069
...,...
2020-12-24 00:00:00-05:00,0.002698
2020-12-28 00:00:00-05:00,0.016988
2020-12-29 00:00:00-05:00,0.000024


In [69]:
# Calculating VaR 
var_95 = np.nanpercentile(pf1_rtn, 5)
var_98 = np.nanpercentile(pf1_rtn, 2)
var_99 =np.nanpercentile(pf1_rtn, 1)

# Parametric VaR
pf1_rtn_mean = pf1_rtn.mean()
p_var_90 = norm.ppf(1 - 0.9, pf1_rtn_mean, pf1_vol)
p_var_98 = norm.ppf(1 - 0.98, pf1_rtn_mean, pf1_vol)

print(var_95)
print(var_98)
print(var_99)
print(p_var_90)
print(p_var_98)

-0.021098740267474842
-0.034313648620933175
-0.04056697513481539
[-0.01724199]
[-0.02847164]


In [70]:
# Calculate Portfolio Return Anualized
pf1_ann_rtn = (df.iloc[-1] - df.iloc[0]) / (df.iloc[0])
pf1_ann_rtn = ((1 + pf1_ann_rtn) ** (12/60))-1
pf1_ann_rtn = pf1_ann_rtn.dot(port_weights)
pf1_ann_rtn

0.27790394526963824

In [71]:
# Calculating Metrics
rf = 0.035
sharpe_ratio  = ((pf1_rtn.mean() * 252 - rf))/(pf1_vol_ann)
sortino = ((pf1_rtn.mean() * 252 - rf)) / (pf1_rtn[pf1_rtn < 0].std() * np.sqrt(252))
mdd = pf1_cum_rtn.expanding(min_periods = 1).max()
mdd = (pf1_cum_rtn/mdd) -1 
mdd = mdd.min()
calmar = (pf1_rtn.mean() * 252)/abs(mdd)

print(sharpe_ratio)
print(sortino)
print(calmar)
print(mdd)


Returns    1.371063
dtype: float64
Returns    1.584276
dtype: float64
Returns    1.279614
dtype: float64
Returns   -0.274704
dtype: float64


In [72]:
# Let's Create a function to calculate all of the metrics
def portfolio_metrics(benchmark, stock_list, dummy_weights, start_date, end_date):

    stock_prices = yf.download(stock_list, start_date, end_date)["Adj Close"]
    benchmark_prices = yf.download(benchmark, start_date, end_date)["Adj Close"]

    if stock_prices.empty:
        return "No data for the selected stock"
    stock_returns = stock_prices.pct_change().dropna()
    benchmark_returns = benchmark_prices.pct_change().dropna()

    benchmark_returns, stock_returns = benchmark_returns.align(stock_returns, join='inner', axis=0)

    portfolio_returns = stock_returns.mul(dummy_weights, axis = 1).sum(axis = 1)

    VaR = norm.ppf(0.05, portfolio_returns.mean(), portfolio_returns.std())

    ES = portfolio_returns[portfolio_returns <= VaR].mean()

    def CVaR(weights):
        portfolio_returns = stock_returns.mul(weights, axis=1).sum(axis=1)
        VaR = norm.ppf(0.05, portfolio_returns.mean(), portfolio_returns.std())
        return -(portfolio_returns[portfolio_returns <= VaR].mean())
    initial_guess = np.ones(len(stock_list)) / len(stock_list)
    bounds = [(0, 1) for _ in range(len(stock_list))]
    CVaR_optimization = minimize(CVaR, initial_guess, bounds=bounds, constraints=({'type': 'eq', 'fun': lambda x: np.sum(x) - 1}))
    CVaR = -CVaR_optimization.fun

    running_max = np.maximum.accumulate(portfolio_returns + 1)
    drawdown = (running_max - (portfolio_returns + 1)) / running_max
    max_drawdown = np.min(drawdown)

    exog = sm.add_constant(benchmark_returns)
    model = sm.OLS(portfolio_returns, exog)
    results = model.fit()
    beta = results.params[1]

    alpha = results.params["const"] - benchmark_returns.mean() * beta

    sharpe_ratio = (portfolio_returns.mean() - benchmark_returns.mean()) / portfolio_returns.std() 

    treynor_ratio = (portfolio_returns.mean() - benchmark_returns.mean()) / beta

    sortino_ratio = (portfolio_returns.mean() - benchmark_returns.mean()) / (portfolio_returns[portfolio_returns < 0].std())

    standard_deviation = portfolio_returns.std()

    covariance = portfolio_returns.corr(benchmark_returns)

    correlation_matrix = stock_returns.corr()

    scenarios = {}
    scenarios["best_case"] = stock_returns.quantile(0.1, axis =1).sum()
    scenarios["worst_case"] = stock_returns.quantile(0.9, axis =1).sum()

    stress_testing = {}
    stress_testing["maximum_loss"] = stock_returns.min().sum()
    stress_testing["maximum_gain"] = stock_returns.max().sum()

    return {
        'VaR': VaR,
        'ES': ES,
        'CVaR': CVaR,
        'Maximum Drawdown': max_drawdown,
        'Beta': beta,
        'Alpha': alpha,
        'Sharpe Ratio': sharpe_ratio,
        'Treynor Ratio': treynor_ratio,
        'Sortino Ratio': sortino_ratio,
        'Standard deviation': standard_deviation,
        'Covariance': covariance,
        'Scenario analysis': scenarios,
        'Stress testing': stress_testing,
        'Correlation matrix': correlation_matrix
    }


In [73]:
# Let's test 
stock_list = ['AAPL', 'MSFT','AMZN', 'NVDA', 'KO', 'MCD', 'NFLX', 'NKE']
benchmark = "^GSPC"
start_date = "2015-01-01" 
end_date = "2020-01-01"
port_weights = np.random.rand(len(stock_list))
port_weights /= np.sum(port_weights)

df3 = portfolio_metrics(benchmark, stock_list, port_weights, start_date, end_date)
df3


[*********************100%***********************]  8 of 8 completed
[*********************100%***********************]  1 of 1 completed


{'VaR': -0.018780926190256723,
 'ES': -0.02871307647946577,
 'CVaR': -0.018374186197723928,
 'Maximum Drawdown': 0.0,
 'Beta': 1.153126240224186,
 'Alpha': 0.0002420647006590308,
 'Sharpe Ratio': 0.0625152029702921,
 'Treynor Ratio': 0.0006569889746313775,
 'Sortino Ratio': 0.08364837292938043,
 'Standard deviation': 0.012118511820963598,
 'Covariance': 0.8055525313771417,
 'Scenario analysis': {'best_case': -13.294123289582608,
  'worst_case': 16.77090769796169},
 'Stress testing': {'maximum_loss': -0.7944347129457341,
  'maximum_gain': 1.057854453907319},
 'Correlation matrix':           AAPL      AMZN        KO       MCD      MSFT      NFLX       NKE  \
 AAPL  1.000000  0.492521  0.236297  0.267730  0.573472  0.367275  0.358001   
 AMZN  0.492521  1.000000  0.214318  0.275732  0.625829  0.482430  0.334932   
 KO    0.236297  0.214318  1.000000  0.388307  0.344771  0.162327  0.297991   
 MCD   0.267730  0.275732  0.388307  1.000000  0.380087  0.174839  0.297340   
 MSFT  0.573472  0.